In [ ]:
import streamlit as st

In [ ]:
st.markdown("""
<div style="background-color: #000020; color: white; text-align: center; padding: 20px">
  <h1 style="margin: 0; color: white"><b>Questioning the Answers: LLMs enter the Boardroom</b></h1>
  <h2 style="margin: 0; color: white"><b></b>Using Gen AI Tools to Harness Alpha from Earnings Calls</h2>
  <h3 style="margin: 0; color: white"><b>by S&P Global Market Intelligence's Quantitative Research & Solutions (QRS) Group</b></h3>
</div>
""", unsafe_allow_html=True)

# 1. Overview
Earnings calls play a pivotal role in shaping investor perceptions. The quality of communication between executives and analysts can significantly influence company performance. On-topic and proactive exeutives, who deliver proactive presentations, anticipate market queries, and provide clear, on-topic answers to analysts’ questions—consistently outperform their peers. Conversely, off-topic and reactive executives, who fail to address analysts’ key inquiries during presentations, and provide off-topic responses—significantly underperform.

Executives' ability to anticipate investor concerns and maintain a focused dialogue fosters confidence and strategic communication. In contrast, failing to provide clarity when analysts seek additional information can lead to misalignment and breakdowns in transparency. A long (short) portfolio of on-topic and proactive (off-topic and reactive) generates +515bps of annualized alpha.

This notebook serves as a introduction for the research detailed in Quantitative Research & Solutions’ recent publication, ["Questioninig the Answers: LLM's enter the Boardroom."](https://www.spglobal.com/market-intelligence/en/news-insights/research/questioning-the-answers-llms-enter-the-boardroom) IIt analyse executive on-topicness and proactiveness using the analysts questions, executives answers and LLM answers. This research harness alpha using LLM tools, including vector embeddings, vector cosine similarity, and the LLM quesiton answering. There is a longer version avalible upon request that also covers how to create the input data from the datasets described in section 2, please reach out to QRS@spglobal.com for access to the longer version.

# 2. Datasets

The ["Questioninig the Answers: LLM's enter the Boardroom."](https://www.spglobal.com/market-intelligence/en/news-insights/research/questioning-the-answers-llms-enter-the-boardroom) research is using the datasets below from the Snowflake Marketplace. Access to those are not neccessary for running this QuickStart, where we are using a sample datase.

To reproduce the full research using the complete datasets then request access to those below using the links or contact SnowflakeMarketplace@spglobal.com.

|Name|Description |
|----|----|
|[ S&P Capital IQ Financials](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D2N/s-p-global-market-intelligence-s-p-capital-iq-financials)|S&P Capital IQ Financials provides global standardized financial statement data for over 180,000 companies, including over 95,000 active and inactive public companies, and As Reported data for over 150,000 companies. S&P Capital IQ Standardized Financials allows you to extend the scope of your historical analysis and back-testing models with consistent data from all filings of a company's historical financial periods including press releases, original filings, and all restatements.|
|[Global Events](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D38/s-p-global-market-intelligence-global-events)|The Global Events dataset provides details on upcoming and past corporate events such as earnings calls, shareholder/analyst meetings, expected earnings release dates and more. With deep history back to 2003, clients can leverage this dataset to derive signals and support trading models across asset classes, trading styles and frequencies. This dataset also helps in research & analysis, risk management & compliance, and trade surveillance workflows.|
|[Machine Readable Transcripts](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D2V/s-p-global-market-intelligence-machine-readable-transcripts)|The Machine Readable Transcripts dataset aggregates data from earnings calls delivered in a machine-readable format for Natural Language Processing (NLP) applications with metadata tagging. Leverage Machine Readable Transcripts to keep track of event information for specific companies including dates, times, dial-in and replay numbers and investor relations contact information. Easily combine data from earnings, M&A, guidance, shareholder, company conference presentations and special calls with traditional datasets to develop proprietary analytics.|
|[Compustat® Financials](https://app.snowflake.com/marketplace/listing/GZT0Z8P3D2R/s-p-global-market-intelligence-compustat®-financials)|Compustat Financials provides standardized North American and global financial statements and market data for over 80,000 active and inactive publicly traded companies that financial professionals have relied on for over 50 years. Compustat allows investment professionals, academic researchers, and industry analysts to combine deep history with robust and consistent data standardization into their research and backtesting to produce valuable insights and generate alpha. With historical data for North America as far back as 1950 and point-in-time snapshots beginning in 1987, Compustat provides you with insight into company financial performance across many different economic cycles not available anywhere else.|

# 3. Libraries & User Inputs
Import libraries required for the workflow

## 3.1 Libraries

Before running, mak sure you have added **cachetools** through **packages**

In [ ]:
# Import python packages
from datetime import datetime
import cachetools

from snowflake.snowpark import functions as snow_funcs
from snowflake.snowpark import Window
from snowflake.snowpark.types import ArrayType, StringType

from snowflake.snowpark.context import get_active_session
session = get_active_session()

## 3.2 User Inputs

This research invloves the usage of an embedding model and a completion model, the default models were set to "snowflake-arctic-embed-m" for embedding and "llama3.1-8b" for completion. This user input section gives you the flexibility to chose your own model for the task.

In [ ]:
# Which of the two embedding functions we want to use
snf_embed_text_func = "SNOWFLAKE.CORTEX.EMBED_TEXT_768" 

# Which embedding model we want to use, see https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability 
# for avalible embedding models in Snowflake
embedding_model = "snowflake-arctic-embed-m" 

# Which LLM we want to use, see https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions#availability 
# for avalible LLMs in Snowflake
completion_model = "llama3.1-8b" 

# Name of the databse created in the Setup Snowflake step
sp_llm_qs_location = "SP_LLM_QS.PUBLIC"


# 4. S&P Global Q4 2024 Earnings Call Transcript

The dataset we are going to use are the prepared remarks, questions and answers sections of the S&P Global Q4 2024 Earnings Call. They have been tokenized on sentence level.

In [ ]:
SELECT * FROM {{sp_llm_qs_location}}.SAMPLE_TRANSCRIPT ORDER BY COMPONENTORDER ASC, SENTENCEORDER ASC LIMIT 50

# 5. Working with the Data: Retrival Augmented Generation (RAG)
Retrieval-Augmented Generation (RAG) is a tool that improves LLM consistency by retrieving relevant information before answering a question.

For this task the LLM needs to be as consistent as possible in its responses to the analysts’ questions as inconsistency will lead to variations in cosine similarity scores and disrupt feature generation downstream.

To combat this, we designed a Retrieval-Augmented Generation (RAG) engine that chunks the prepared remarks sentence-by-sentence and retrieves the optimal retrieval percentage of sentences most similar to the question. Inconsistency occurs when the LLM is provided with too little (or too much) context, it becomes uninformed (unspecific). The optimal retrieval percentage for consistency is 60%.

In the cells below, we vector embed all questions, prepared remark sentences and answer sentences using the snowflake-arctic-m embedding model. Then use the cosine similarity from the (question vs prepared remark sentences) and (question vs answer sentences) to select top 60% most relevant prepared remark and answer sentences to the question from S&P Global Q4 2024 Earnings Call Transcript

## 5.1 S&P Global Q4 2024 Earnings Call Transcript
Select and seperate the prepared remarks, questions and answers sections of the S&P Global Q4 2024 Earnings Call

In [ ]:
all_component_df = session.table(f"{sp_llm_qs_location}.SAMPLE_TRANSCRIPT")

pppSentences = all_component_df.filter(snow_funcs.col('transcriptComponentTypeId') == 2) 
questions = all_component_df.filter(snow_funcs.col('transcriptComponentTypeId') == 3)
answerSentences = all_component_df.filter(snow_funcs.col('transcriptComponentTypeId') == 4) 

st.dataframe(pppSentences.limit(5))
st.dataframe(questions.limit(5))
st.dataframe(answerSentences.limit(5))

## 5.2 Vector Embedding
Transform the questions, prepared remark sentences and answer sentences into numerical
representations using the snowflake-arctic-m embedding model.

### 5.2.2 Apply Embedding to Transcript Components

In [ ]:
snf_embed_text = snow_funcs.function(snf_embed_text_func)

pppSentences_with_embeddings = pppSentences.withColumn("sentenceVec", snf_embed_text(snow_funcs.lit(embedding_model)
                                                                                     ,pppSentences["PROCESSEDTEXT"]))
questions_with_embeddings = questions.withColumn("questionVec", snf_embed_text(snow_funcs.lit(embedding_model),
                                                                                    questions["PROCESSEDTEXT"]))
answerSentences_with_embeddings = answerSentences.withColumn("sentenceVec",  snf_embed_text(snow_funcs.lit(embedding_model)
                                                                                            ,answerSentences["PROCESSEDTEXT"]))

pppSentences_with_embeddings.write.save_as_table(f"{sp_llm_qs_location}.pppSentencesVec", mode="overwrite",table_type='transient')
questions_with_embeddings.write.save_as_table(f"{sp_llm_qs_location}.questionsVec", mode="overwrite", table_type='transient')
answerSentences_with_embeddings.write.save_as_table(f"{sp_llm_qs_location}.answerSentencesVec", mode="overwrite", table_type='transient')

# 5.3 Cosine Similarity
To determine semantic closeness, we vector-embed the question-and-answer texts and calculate a cosine similarity score between the two vectors.
For example, A and B each represent a vector such that:


$$	A = [a_1,a_2,… a_n] $$
$$	B = [b_1,b_2,… b_n] $$


The cosine similarity formula between vectors A and B is:

$$\text{Cosine Similarity} = \frac{\mathbf{A} \cdot \mathbf{B}}{\|\mathbf{A}\| \cdot \|\mathbf{B}\|}$$

, where A⋅B is the dot product of the vectors, and |A|⋅|B| is the product of each vector's magnitude.
The result ranges from -1 to 1, where:

	-1: Vectors are opposite.
	0: Vectors are unrelated.
	1: Vectors are identical.

### 5.3.2 Apply Cosine Similarity to Question & Answer Vector Embeddings
The cell below creates question and answer pair by collecting all the answer sentences whose componentOrder is between the currentQuestionComponentOrder and the nextQuestionComponentOrder. Then the the consine similarity was applied to the question and answer sentence vector embeddings.

In [ ]:
df = session.sql(f"""

with q as(
    select 
        callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter,
        tradingItemId, companyId, companyName, headline, transcriptId,
        speakerTypeName as questionSpeakerTypeName, transcriptPersonName as questionTranscriptPersonName,
        transcriptPersonId as questionTranscriptPersonId, proId as questionProId, transcriptComponentTypeId as questionTranscriptComponentTypeId,
        transcriptComponentId as questionTranscriptComponentId, componentOrder as currentQuestionOrder
        , lead(componentOrder) over (partition by tradingItemId, transcriptId order by componentOrder) as nextQuestionOrder
        , processedText as question, questionVec
    from {sp_llm_qs_location}.questionsVec
    where 1=1
        and transcriptComponentTypeId = 3
)
select 
    q.callDate, q.enteredDate, q.fiscalYearQuarter, q.calendarYearQuarter,
    q.tradingItemId, q.companyId, q.companyName, q.headline, q.transcriptId,
    q.questionSpeakerTypeName, q.questionTranscriptPersonName, q.questionTranscriptPersonId, q.questionProId, 
    a.speakerTypeName as answerSpeakerTypeName, a.transcriptPersonName as answerTranscriptPersonName, a.transcriptPersonId as answerTranscriptPersonId, a.proId as answerProId, 
    q.questionTranscriptComponentTypeId, a.transcriptComponentTypeId as answerTranscriptComponentTypeId,
    q.questionTranscriptComponentId, a.transcriptComponentId as answerTranscriptComponentId,
    q.currentQuestionOrder, q.nextQuestionOrder, a.componentOrder as answerOrder, a.sentenceOrder as answerSentenceOrder,
    q.question, a.componentText as answer, a.processedText as answerSentence, q.questionVec, a.sentenceVec as answerSentenceVec
    , VECTOR_COSINE_SIMILARITY(questionVec, sentenceVec) AS cosine_similarity
from q
    join {sp_llm_qs_location}.answerSentencesVec a on q.tradingItemId = a.tradingItemId and q.transcriptId = a.transcriptId and a.componentOrder between q.currentQuestionOrder and ifnull(q.nextQuestionOrder, 10000)
order by q.nextQuestionOrder asc, a.componentOrder asc, a.sentenceOrder asc

""")

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_qaPairCos", mode="overwrite", table_type='transient')
df.limit(5)

### 5.3.3 Apply Cosine Similarity to Question & Prepared Remarks Vector Embeddings
The cell below pairs all prepared remarks sentences to questions. Then the the consine similarity was applied to the question and prepared remarks sentence vector embeddings.

In [ ]:
df = session.sql(f"""

with q as(
    select 
        callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter,
        tradingItemId, companyId, companyName, headline, transcriptId,
        speakerTypeName as questionSpeakerTypeName, transcriptPersonName as questionTranscriptPersonName,
        transcriptPersonId as questionTranscriptPersonId, proId as questionProId, transcriptComponentTypeId as questionTranscriptComponentTypeId,
        transcriptComponentId as questionTranscriptComponentId, componentOrder as currentQuestionOrder, lead(componentOrder) over (partition by tradingItemId, transcriptId order by componentOrder) as nextQuestionOrder, 
        processedText as question, questionVec
    from {sp_llm_qs_location}.questionsVec
    where 1=1
        and transcriptComponentTypeId = 3
)
select 
    q.*, 
    p.componentOrder as executiveRemarkComponentOrder, p.sentenceOrder as executiveRemarkSentenceOrder,
    p.componentText as executiveRemark, p.processedText as executiveSentence, p.sentenceVec as executiveVec,
    VECTOR_COSINE_SIMILARITY(questionVec, p.sentenceVec) as similarity
from q
    join {sp_llm_qs_location}.pppSentencesVec p on q.tradingItemId = p.tradingItemId and q.transcriptId = p.transcriptId

order by q.nextQuestionOrder asc, p.componentOrder asc, p.sentenceOrder asc

""")

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_pppQPairCos", mode="overwrite", table_type='transient')
df.limit(5)

## 5.4 Top 60% Sentences
Utilizing the top 60% of prepared remarks identified as generating the most consistent LLM output. For further details on the experiment, please refer to the 'LLM Robustness Check' section in the whitepaper.

### 5.4.2 Concat Top 60% Answer Sentences
After selecting the top 60% most similar answer sentences, we concat the answer sentences on the question level.

In [ ]:

df = session.sql(f'''
          
with qaSimilarityRank as(
    select 
        *, 
        ROW_NUMBER() OVER (PARTITION BY tradingItemId, transcriptId, currentQuestionOrder, currentQuestionOrder ORDER BY cosine_similarity desc) AS similarityRank,
        COUNT(*) OVER (PARTITION BY tradingItemId, transcriptId, currentQuestionOrder, currentQuestionOrder) AS answerSentencesCount 
    from {sp_llm_qs_location}.TranscriptComponents_qaPairCos 
    ), 
    questionWithSixtyPercentAnswers as(

    select 
        *, 
        case 
            when answerSentencesCount = 1 then 1
            when similarityRank <= answerSentencesCount * 0.67 then 1
            else 0
        end as toKeepFlag
    from qaSimilarityRank

    ),
    qaPair60 as(
    select
        callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName
        , headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId
        , questionProId, answerSpeakerTypeName, answerTranscriptPersonName, answerTranscriptPersonId, answerProId
        , questionTranscriptComponentTypeId, answerTranscriptComponentTypeId, questionTranscriptComponentId
        , answerTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, answerOrder, question, answer, 
        ARRAY_TO_STRING(ARRAY_AGG(answerSentence), ' ') AS sixtyPercentAnswer
    from (select * from questionWithSixtyPercentAnswers where toKeepFlag = 1 order by currentQuestionOrder asc, answerOrder asc, answerSentenceOrder asc)
    group by callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, answerSpeakerTypeName, answerTranscriptPersonName, answerTranscriptPersonId, answerProId, questionTranscriptComponentTypeId, answerTranscriptComponentTypeId, questionTranscriptComponentId, answerTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, answerOrder, question, answer)
    select 
        callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, answerSpeakerTypeName, answerTranscriptPersonName, answerTranscriptPersonId, answerProId, questionTranscriptComponentTypeId, answerTranscriptComponentTypeId, questionTranscriptComponentId, answerTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, answerOrder, 
        question, answer
        , SNOWFLAKE.CORTEX.COUNT_TOKENS('{completion_model}',answer) as answerTokenCount
        , sixtyPercentAnswer
        , SNOWFLAKE.CORTEX.COUNT_TOKENS('{completion_model}',sixtyPercentAnswer) as sixtyPercentAnswerTokenCount
    from qaPair60
    order by currentQuestionOrder asc, answerOrder asc
              
''')

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_qaPairTop60", mode="overwrite", table_type='transient')
df.limit(5)

### 5.4.3 Concat Top 60% Prepared Remarks Sentences
Similarly, after selecting the top 60% most similar prepared remarks sentences, we concat the prepared remarks sentences on the question level.

In [ ]:

df = session.sql(f'''

with qaSimilarityRank as(
select 
    *, 
    ROW_NUMBER() OVER (PARTITION BY tradingItemId, transcriptId, currentQuestionOrder ORDER BY similarity desc) AS similarityRank,
    COUNT(*) OVER (PARTITION BY tradingItemId, transcriptId, currentQuestionOrder) AS executiveRemarksSentencesCount
from {sp_llm_qs_location}.TranscriptComponents_pppQPairCos
),

pppQPairTop60 as(
select 
    *, 
    case 
        when executiveRemarksSentencesCount = 1 then 1
        when similarityRank <= executiveRemarksSentencesCount * 0.67 then 1
        else 0
    end as toKeepFlag
from qaSimilarityRank),

presenterLevelSixtyPercentPPP as(
select
    callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, executiveRemarkComponentOrder, question, executiveRemark, 
    ARRAY_TO_STRING(ARRAY_AGG(executiveSentence), ' ') AS sixtyPercentExecutiveRemark
from (select * from pppQPairTop60 where toKeepFlag = 1 order by currentQuestionOrder asc, executiveRemarkComponentOrder asc, executiveRemarkSentenceOrder asc)
group by callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, executiveRemarkComponentOrder, question, executiveRemark
order by currentQuestionOrder asc,  executiveRemarkComponentOrder asc),

pppQPairTop60Concat as(
select 

    callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, question,
    ARRAY_TO_STRING(ARRAY_AGG(executiveRemark), '\n\n') AS questionLevelConcatenatedRawExecutiveRemark,
    ARRAY_TO_STRING(ARRAY_AGG(sixtyPercentExecutiveRemark), '\n\n') AS questionLevelConcatenatedSixtyPercentExecutiveRemark
from (select * from presenterLevelSixtyPercentPPP order by currentQuestionOrder asc,  executiveRemarkComponentOrder asc)
group by callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, question
)

select 
    callDate, enteredDate, fiscalYearQuarter, calendarYearQuarter, tradingItemId, companyId, companyName, headline, transcriptId, questionSpeakerTypeName, questionTranscriptPersonName, questionTranscriptPersonId, questionProId, questionTranscriptComponentTypeId, questionTranscriptComponentId, currentQuestionOrder, nextQuestionOrder, question,
    questionLevelConcatenatedRawExecutiveRemark
    , SNOWFLAKE.CORTEX.COUNT_TOKENS('{completion_model}',questionLevelConcatenatedRawExecutiveRemark) as questionLevelConcatenatedRawExecutiveRemarkTokenCount, questionLevelConcatenatedSixtyPercentExecutiveRemark
    , SNOWFLAKE.CORTEX.COUNT_TOKENS('{completion_model}',questionLevelConcatenatedSixtyPercentExecutiveRemark) as questionLevelConcatenatedSixtyPercentExecutiveRemarkTokenCount
from pppQPairTop60Concat
order by currentQuestionOrder asc

''')

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_pppQPairTop60", mode="overwrite", table_type='transient')
df.limit(5)

# 6. Working with the Data: LLM Ready Data

Using a LLM to answer analysts questions based only on the prepared remarks and the previous questions and answers will give an indication if executives are proactive.

## 6.1 Using Snowflake Cortex AI
When calling the Snowflake Cortex COMPLETE function, messages are organized into distinct roles—system, user, and assistant—to structure and guide interactions. Each role serves a specific purpose:

System: Provides instructions that define the context or behavior of the model. It's like setting the rules or tone for the conversation. Example: "You are a helpful assistant that answers questions about technology in a concise manner."

User: Represents the input or queries made by the person interacting with the model. These are the prompts or requests that the model responds to. Example: "What is the purpose of the OpenAI API?"

Assistant: Reflects the model's response to the user's query, shaped by the system's instructions and the user's input. Example: "The OpenAI API is designed to enable developers to integrate language models into their applications for tasks like answering questions, generating content, and more."

In our research, executive prepared remarks are labelled as assistant messages, analyst's questions as User messages and executive answers as Assistant messages

### 6.1.1 Construct COMPLETE Assistant Message with QA Pair Snippet

In [ ]:
df = session.sql(f'''

with questionWithSixtyPercentAnswersPromptSnippet as (
select
    *, 
    OBJECT_CONSTRUCT('role', 'user', 'content', REPLACE(question, '\r', '')) as questionPromptSnippet,
    OBJECT_CONSTRUCT('role', 'assistant', 'content', REPLACE(sixtyPercentAnswer, '\r', '')) as answerPromptSnippet
    
from {sp_llm_qs_location}.TranscriptComponents_qaPairTop60)
select 
    *, ARRAY_CONSTRUCT(questionPromptSnippet, answerPromptSnippet) as questionAnswerPairPromptSnippet 
from questionWithSixtyPercentAnswersPromptSnippet
order by currentQuestionOrder asc, answerOrder asc

''')


df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_pppQPairTop60AnswerConcat", mode="overwrite", table_type='transient')
df.limit(5)


### 6.1.2 Construct Snowflake Cortex COMPLETE Assistant Message with Prepared Remarks Snippets

In [ ]:
df = session.sql(f'''

select
    *,
    OBJECT_CONSTRUCT('role', 'user', 'content', REPLACE(question, '\r', '')) as questionPromptSnippet,
    OBJECT_CONSTRUCT('role', 'assistant', 'content', REPLACE(questionLevelConcatenatedSixtyPercentExecutiveRemark, '\r', '')) as pppPromptSnippet
from {sp_llm_qs_location}.TranscriptComponents_pppQPairTop60

''')

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_pppQPairTop60PPPConcat", mode="overwrite", table_type='transient')
df.limit(5)

## 6.2 Collect All Messages for Cortex COMPLETE
All question pairs with prepare remarks and answers come together to form an LLM prompt following the iterative process such that:

1. 'user': 'From the perspective of a top executive, please answer the following question raised by a financial analyst during an earnings conference call. Knowledge cutoff date: '  
2. 'assistant': 60% prepared remarks  
3. 'user': question 1  
4. 'assistant': 60% answer 1  
5. ...  
6. ...  
7. 'user': question n  

### 6.2.1 LLM Ready Prompt Messages
In the dataframe below, the prompt column has all the messages in 1 list. This is the prompt for the LLM.

In [ ]:
df = session.sql(f'''
    with pppQPairTop60PPPConcat
    as
    (
    select 
         a.callDate, a.tradingItemId, a.transcriptId, a.headline, 
        a.questionTranscriptPersonName, a.questionTranscriptPersonId, a.questionProId,
        a.answerTranscriptPersonName, a.answerTranscriptPersonId, a.answerProId,
        a.questionTranscriptComponentId, a.answerTranscriptComponentId,
        a.question, a.answer,
        ARRAY_CONSTRUCT(object_construct('role', 'user', 'content', concat('From the perspective of a top executive, please answer the following question raised by a financial analyst during an earnings conference call. Knowledge cutoff date: ', cast(a.callDate as string)))) as initPrompt,
       a.questionPromptSnippet,
       ARRAY_FLATTEN(ARRAY_AGG(a.questionAnswerPairPromptSnippet) OVER (PARTITION BY a.TRANSCRIPTID ORDER BY a.CURRENTQUESTIONORDER 
                                                        ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING)) as concatenatedPredecessors
     from {sp_llm_qs_location}.TranscriptComponents_pppQPairTop60AnswerConcat a
    )
    select a.* 
        , ARRAY_CAT(a.initPrompt, ARRAY_CAT([ppp.pppPromptSnippet], ARRAY_CAT(a.concatenatedPredecessors, [a.questionPromptSnippet]))) as prompt
    from 
    pppQPairTop60PPPConcat a
    join {sp_llm_qs_location}.TranscriptComponents_pppQPairTop60PPPConcat as ppp on a.tradingItemId = ppp.tradingItemId and a.transcriptId = ppp.transcriptId and a.questionTranscriptComponentId = ppp.questionTranscriptComponentId
    order by a.questionTranscriptComponentId asc, a.answerTranscriptComponentId asc
''')

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_targetLLMReadyPrompt", mode="overwrite", table_type='transient')
df.limit(5)


## 6.3 Collect LLM Response

### 6.3.1 Apply LLM Completion
We apply the SNOWFLAKE.CORTEX.COMPLETE function on the prompt column using the model defined by `completion_model`and collect the LLM response.

In [ ]:
df = session.sql(f"""

    select *,  SNOWFLAKE.CORTEX.COMPLETE('{completion_model}', prompt, {{'temperature': 0}}) as LLMAnswer 
    from {sp_llm_qs_location}.TranscriptComponents_targetLLMReadyPrompt
          
""")

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_targetLLMAnswer", mode="overwrite", table_type='transient')
df.limit(5)

### 6.3.2 Clean Up LLM Response

Extract only the actual message from the LLM from the responses

In [ ]:
df = session.sql(f"""

    select 
        callDate, tradingItemId, transcriptId, headline, 
        questionTranscriptPersonName, questionTranscriptPersonId, questionProId, answerTranscriptPersonName, answerTranscriptPersonId, answerProId,
        questionTranscriptComponentId, answerTranscriptComponentId, question, answer,
        REPLACE(LLMANSWER: "choices"[0]: "messages", '"', '') as cleanLLMAnswer 
    from {sp_llm_qs_location}.TranscriptComponents_targetLLMAnswer
          
""")

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_targetLLMAnswer_Clean", mode="overwrite", table_type='transient')
df.limit(5)

## 6.4 Summarize Text

Use the Snowflake Cortex Summarize function to summarize the question, answer and LLM answer

In [ ]:
df = session.sql(f"""

    select 
        *, 
        SNOWFLAKE.CORTEX.SUMMARIZE(question) as summarizeQuestion,
        SNOWFLAKE.CORTEX.SUMMARIZE(answer) as summarizeAnswer,
        SNOWFLAKE.CORTEX.SUMMARIZE(cleanLLMAnswer) as summarizeCleanLLMAnswer,
    from {sp_llm_qs_location}.TranscriptComponents_targetLLMAnswer_Clean
          
""")

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_targetLLMAnswer_Summarize", mode="overwrite", table_type='transient')
df.limit(5)

# 7. Working with the Data: Factor Construction
## 7.1 Executive On/Off Topic Factor
When an executive answer is semantically similar (dissimilar) to the analyst’s question, it suggests that the answer uses language and concepts similar to (different from) the analyst question, indicating it is on-topic (off-topic). To determine semantic closeness, we vector-embed the question-and-answer texts and calculate a cosine similarity score between the two vectors.

### 7.1.1 Question vs Executive Answer Cosine Similarity

In [ ]:
df = session.sql(f'''

    with vec as(
    select *
        , {snf_embed_text_func}('{embedding_model}', summarizeQuestion) as questionVec
        , {snf_embed_text_func}('{embedding_model}', summarizeAnswer) as answerVec 
    from {sp_llm_qs_location}.TranscriptComponents_targetLLMAnswer_Summarize
    )
    select *, VECTOR_COSINE_SIMILARITY(questionVec, answerVec) as execOnOffTopicFactor from vec 
               
''')

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_execOnOffTopicFactor", mode="overwrite", table_type='transient')
df.limit(5)

### 7.1.2 Transcript Mean Executive On/Off Topic Factor
Cosine similarity scores are averaged at the transcript level. A high (low) Cosine Similarity Score indicates an On (Off) Topic Executive.



In [ ]:
select avg(execOnOffTopicFactor) as transcriptLevelExecOnOffTopicFactor 
from {{sp_llm_qs_location}}.TranscriptComponents_execOnOffTopicFactor

## 7.2 Executive Proactive/Reactive Factor
### 7.2.1 Question vs LLM Answer Cosine Similarity
Since the LLM answers only within the context of information provided in the prepared remarks, a high (low) cosine similarity score indicates that the LLM answers are semantically similar (dissimilar) the questions, reflecting the executives are proactive (reactive).

In [ ]:
df = session.sql(f'''

    with vec as(
    select 
        *, 
        {snf_embed_text_func}('{embedding_model}', summarizeQuestion) as questionVec, 
        {snf_embed_text_func}('{embedding_model}', summarizeCleanLLMAnswer) as LLMAnswerVec 
    from {sp_llm_qs_location}.TranscriptComponents_targetLLMAnswer_Summarize
    )
    select *, VECTOR_COSINE_SIMILARITY(questionVec, LLMAnswerVec) as execProactiveReactiveFactor from vec 
               
''')

df.write.save_as_table(f"{sp_llm_qs_location}.TranscriptComponents_execProactiveReactiveFactor", mode="overwrite", table_type='transient')
df.limit(5)


### 7.2.2 Transcript Mean Executive Proactive/Reactive Factor
Similar to the construction of the Executive On/Off Topic factor, both the LLM answers and questions are summarized, vector-embedded and cosine similarity scores are averaged at the transcript level.

In [ ]:
select avg(execProactiveReactiveFactor) as transcriptLevelexecProactiveReactiveFactor 
from {{sp_llm_qs_location}}.TranscriptComponents_execProactiveReactiveFactor

# 8. Results & Summary
This research underscores the significant impact of executive communication styles during earnings calls on firm performance. Proactive executives who anticipate market concerns and provide concise, on-topic responses foster transparency, aligning with investor expectations and driving superior returns. The findings demonstrate that firms with Efficient Communicators achieve statistically significant outperformance, while Total Redirectors suffer from diminished confidence and underperformance. These insights validate the critical role of strategic communication in shaping investor perceptions and influencing market outcomes.

Advanced analytical tools, such as vector embeddings and cosine similarity metrics, enable nuanced evaluations of executive-analyst interactions, revealing measurable performance effects across different communication styles. While large language models (LLMs) enhance feature extraction, challenges like forward-looking bias and inconsistency highlight the need for caution in time-sensitive tasks. Overall, the integration of proactive, clear, and relevant communication strategies remains paramount in fostering investor trust and maximizing financial success in a competitive marketplace.